In [1]:
# 1️⃣ Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score
from sentence_transformers import SentenceTransformer
import joblib
import os

# 2️⃣ Read dataset
df = pd.read_csv("HEAR_Dataset.csv")  # adjust path if needed
list_of_texts = df["Review Text"].tolist()
labels_array = df[["Pricing", "Appointments", "Medical Staff", "Customer Service", "Emergency Services"]].values

# 3️⃣ Split train/test
texts_train, texts_test, labels_train, labels_test = train_test_split(
    list_of_texts, labels_array, test_size=0.2, random_state=42
)

In [2]:

# 4️⃣ Sentence-BERT Embeddings
# Using AraBERT-based sentence embedding model
embedder = SentenceTransformer('aubmindlab/bert-base-arabertv2')  # GPU-enabled if available
train_embeddings = embedder.encode(texts_train, batch_size=16, show_progress_bar=True)
test_embeddings = embedder.encode(texts_test, batch_size=16, show_progress_bar=True)

# 5️⃣ Convert labels
# Labels are already integers [0,1,2,3], no conversion needed
labels_train_array = np.array(labels_train)
labels_test_array = np.array(labels_test)

# 6️⃣ Dataset Class (optional for ML, here just for structure)
class FeedbackDataset:
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

train_dataset = FeedbackDataset(train_embeddings, labels_train_array)
test_dataset = FeedbackDataset(test_embeddings, labels_test_array)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Batches:   0%|          | 0/1151 [00:00<?, ?it/s]

Batches:   0%|          | 0/288 [00:00<?, ?it/s]

In [3]:
# 7️⃣ Model Definition
# Multi-output Logistic Regression (one classifier per aspect)
aspect_names = ["Pricing", "Appointments", "Medical Staff", "Customer Service", "Emergency Services"]
models = [LogisticRegression(max_iter=500, multi_class='multinomial', solver='lbfgs') for _ in aspect_names]

# 8️⃣ Training Setup
# Already minimal for ML, nothing else needed

# 9️⃣ Training Loop
for i, model in enumerate(models):
    y_train_aspect = labels_train_array[:, i]
    model.fit(train_embeddings, y_train_aspect)
    print(f"✅ Trained classifier for aspect: {aspect_names[i]}")

# 10️⃣ Saving the Model
save_dir = "./saved_model_sbert"
os.makedirs(save_dir, exist_ok=True)
for i, model in enumerate(models):
    joblib.dump(model, os.path.join(save_dir, f"{aspect_names[i]}_clf.joblib"))
# Save embedder as well
embedder.save(os.path.join(save_dir, "sentence_transformer"))
print(f"✅ All models and embedder saved to: {save_dir}")

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


✅ Trained classifier for aspect: Pricing


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


✅ Trained classifier for aspect: Appointments


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


✅ Trained classifier for aspect: Medical Staff


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


✅ Trained classifier for aspect: Customer Service


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


✅ Trained classifier for aspect: Emergency Services
✅ All models and embedder saved to: ./saved_model_sbert


In [4]:
# 11️⃣ Evaluation / Testing
all_preds = [[] for _ in range(5)]
all_labels = [[] for _ in range(5)]
all_probs = [[] for _ in range(5)]

for i, model in enumerate(models):
    y_test_aspect = labels_test_array[:, i]
    preds = model.predict(test_embeddings)
    probs = model.predict_proba(test_embeddings)

    all_preds[i] = preds
    all_labels[i] = y_test_aspect
    all_probs[i] = probs

metrics = []
for i, aspect in enumerate(aspect_names):
    y_true = np.array(all_labels[i])
    y_pred = np.array(all_preds[i])
    y_prob = np.array(all_probs[i])

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    # Compute mean Average Precision (mAP)
    y_true_onehot = np.zeros_like(y_prob)
    y_true_onehot[np.arange(len(y_true)), y_true] = 1
    try:
        mAP = average_precision_score(y_true_onehot, y_prob, average='macro')
    except ValueError:
        mAP = 0.0  # in case only one class appears

    metrics.append({
        "Aspect": aspect,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1,
        "mAP": mAP
    })

results_path = os.path.join("results", "evaluation_results_sbert.txt")
os.makedirs("results", exist_ok=True)
with open(results_path, "w", encoding="utf-8") as f:
    f.write("📊 Evaluation Results\n")
    f.write("="*50 + "\n")
    for m in metrics:
        f.write(
            f"{m['Aspect']}\n"
            f"  Accuracy: {m['Accuracy']:.4f}\n"
            f"  Precision: {m['Precision']:.4f}\n"
            f"  Recall: {m['Recall']:.4f}\n"
            f"  F1-score: {m['F1']:.4f}\n"
            f"  mAP: {m['mAP']:.4f}\n"
            + "-"*50 + "\n"
        )
    mean_f1 = np.mean([m['F1'] for m in metrics])
    mean_map = np.mean([m['mAP'] for m in metrics])
    f.write(f"\nOverall Mean F1: {mean_f1:.4f}\n")
    f.write(f"Overall Mean mAP: {mean_map:.4f}\n")
print(f"✅ Evaluation completed. Results saved to: {results_path}")

✅ Evaluation completed. Results saved to: results/evaluation_results_sbert.txt


In [6]:
# 10.5️⃣ Zip and download the saved models
import shutil
from google.colab import files

zip_path = shutil.make_archive("saved_model_sbert", 'zip', save_dir)
print(f"📦 Model folder zipped at: {zip_path}")

# Start download
files.download(zip_path)
print("⬇️ Download started! Check your browser’s download bar.")


📦 Model folder zipped at: /content/saved_model_sbert.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ Download started! Check your browser’s download bar.


In [5]:

# 12️⃣ How to Use the Model Locally
# Load models and embedder
loaded_models = [joblib.load(os.path.join(save_dir, f"{aspect}_clf.joblib")) for aspect in aspect_names]
loaded_embedder = SentenceTransformer(os.path.join(save_dir, "sentence_transformer"))

# Example prediction
example_review = ["الخدمة ممتازة لكن الأسعار مرتفعة"]
example_embedding = loaded_embedder.encode(example_review)
example_preds = [model.predict(example_embedding)[0] for model in loaded_models]
print(f"Example review predictions per aspect: {dict(zip(aspect_names, example_preds))}")

Example review predictions per aspect: {'Pricing': np.int64(0), 'Appointments': np.int64(3), 'Medical Staff': np.int64(3), 'Customer Service': np.int64(1), 'Emergency Services': np.int64(3)}
